<a href="https://colab.research.google.com/github/dimageneer/AIEFFEL_QUEST/blob/main/%EC%9E%91%EC%82%AC%EA%B0%80_L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
sentence = " 나는 밥을 먹었다 "
# <start>는 문장의 시작 입니다.
# <start> 토큰을 받은 순환 신경망은 "나는"을 출력한다.
# 출력된 "나는"이라는 단어를 다시 입력으로 사용하고 이러한 반복을 통해 "먹었다"까지
# 글을 생성한다. 마지막으로 끝(완성)을 뜻하는 <end> 토큰을 생성하여 마무리 합니다.
source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
# Source 문장: <start> 나는 밥을 먹었다
print("Target 문장:", target_sentence)
# Target 문장:  나는 밥을 먹었다 <end>

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# os: Operating System의 줄임말로, 운영체제에서 제공되는 여러 기능을 파이썬에서 사용할 수 있도록 함 (Ex. 디렉토리 경로 이동, 시스템 환경 변수 가져오기 등)
# re: Regular Expression의 줄임말로, 파이썬 정규표현식을 사용하기 위한 모듈
# numpy(NumPy): 행렬이나 대규모 다차원 배열을 쉽게 처리할 수 있도록 지원하는 라이브러리. 데이터 구조 외에도 수치 계산을 위해 효율적으로 구현된 기능을 제공
# tensorflow(TensorFlow): 데이터 흐름 프로그래밍을 위한 라이브러리. 데이터 그래프를 수치적으로 연산하기 위해 구글에서 만든 오픈 소스 라이브러리.
import os, re
import numpy as np
import tensorflow as tf

# os 모듈에 있는 getenv() 함수를 이용하여 읽고싶은 파일의 경로를 file_path에 저장
# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = '/content/drive/MyDrive/Colab Notebooks/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [10]:
# enumerate() 함수를 이용하여 raw_corpus list 내에 저장된 문장과 그 문장의 인덱스를 반환 (인덱스, 문장 순)
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.

    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [11]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [12]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [13]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras.src.legacy.preprocessing.text.Tokenizer object at 0x7df7461dc7a0>


In [14]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [15]:
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [16]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0]


In [17]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 7000개로 선언했기 때문에, tokenizer.num_words의 값은 7000
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(256, 19), dtype=tf.int32, name=None), TensorSpec(shape=(256, 19), dtype=tf.int32, name=None))>

In [18]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다.
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [19]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 19, 7001), dtype=float32, numpy=
array([[[ 2.43132803e-04,  2.02579540e-05, -7.06462160e-05, ...,
         -3.56827622e-05,  2.60386500e-04, -6.01153879e-05],
        [ 2.88625422e-04,  2.65144263e-06, -8.38097330e-05, ...,
         -1.93936328e-04,  2.78140476e-04,  3.55479096e-05],
        [ 5.99558698e-04, -1.55803878e-04, -3.25996429e-04, ...,
          1.06435691e-04,  6.52052171e-04,  1.25934472e-04],
        ...,
        [ 4.41510463e-03, -6.36347989e-03,  2.72911944e-04, ...,
         -1.21863268e-03, -2.70615984e-03, -1.43314223e-03],
        [ 4.46057739e-03, -6.60593761e-03,  2.68680131e-04, ...,
         -1.34125655e-03, -2.87834066e-03, -1.41933642e-03],
        [ 4.49473457e-03, -6.80298964e-03,  2.59887776e-04, ...,
         -1.44801580e-03, -3.02946474e-03, -1.39464485e-03]],

       [[ 2.43132803e-04,  2.02579540e-05, -7.06462160e-05, ...,
         -3.56827622e-05,  2.60386500e-04, -6.01153879e-05],
        [ 5.59171487e-04,  4.49736777e-04,  1.

In [20]:
# 모델의 구조를 확인합니다.
model.summary()

Model: "text_generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (256, 19, 256)         │     1,792,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (256, 19, 1024)        │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (256, 19, 1024)        │     8,392,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (256, 19, 7001)        │     7,176,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,607,961 (86.24 MB)

 Trainable params: 22,607,961 (86.24 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다

# Adam 알고리즘을 구현하는 optimzier이며 어떤 optimzier를 써야할지 모른다면 Adam을 쓰는 것도 방법이다.
# 우리가 학습을 할 때 최대한 틀리지 않는 방향으로 학습을 해야한다.
# 여기서 얼마나 틀리는지(loss)를 알게하는 함수가 손실함수 이다.
# 이 손실함수의 최소값을 찾는 것을 학습의 목표로 하며 여기서 최소값을 찾아가는 과정을 optimization 이라하고
# 이를 수행하는 알고리즘을 optimizer(최적화)라고 한다.

optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다.
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
model.fit(dataset, epochs=30) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

Epoch 1/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 31s 301ms/step - loss: 4.3780
Epoch 2/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 30s 324ms/step - loss: 2.8921
Epoch 3/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 364ms/step - loss: 2.7325
Epoch 4/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 362ms/step - loss: 2.6465
Epoch 5/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 33s 354ms/step - loss: 2.5737
Epoch 6/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 364ms/step - loss: 2.5055
Epoch 7/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 33s 359ms/step - loss: 2.4401
Epoch 8/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 360ms/step - loss: 2.4049
Epoch 9/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 363ms/step - loss: 2.3485
Epoch 10/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 363ms/step - loss: 2.2883
Epoch 11/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 361ms/step - loss: 2.2331
Epoch 12/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 362ms/step - loss: 2.1775
Epoch 13/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 363ms/step - loss: 2.1223
Epoch 14/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 362ms/step - loss: 2.0550
Epoch 15/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 

In [24]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [23]:
generate_text(model, tokenizer, init_sentence="<start> he") # 시작문장으로 he를 넣어 문장생성 함수 실행

'<start> he s a changeling and an old <unk> <end> '

# Task
이 노트북을 통해 셰익스피어 스타일의 문장을 생성하는 AI 작문 모델을 성공적으로 구현했는지 확인합니다. 이 모델은 주어진 단어를 시작으로 자연스러운 문장을 이어서 쓸 수 있습니다.

## 데이터 로드 및 초기 전처리

### Subtask:
Google Drive에 있는 `shakespeare.txt` 파일을 불러와 `raw_corpus`에 저장하고, 빈 줄이나 콜론으로 끝나는 줄 등 불필요한 문장을 초기 필터링합니다.


**Reasoning**:
To confirm that sentences with length 0 or ending with a colon (:) were excluded from the corpus, I will display a few initial sentences from `raw_corpus` and then from the `corpus` variable, which has undergone this initial filtering.



In [25]:
print("--- Raw Corpus (first 15 sentences) ---")
for i, sentence in enumerate(raw_corpus[:15]):
    print(f"{i}: {sentence}")

print("\n--- Filtered Corpus (first 10 sentences) ---")
for i, sentence in enumerate(corpus[:10]):
    print(f"{i}: {sentence}")

--- Raw Corpus (first 15 sentences) ---
0: First Citizen:
1: Before we proceed any further, hear me speak.
2: 
3: All:
4: Speak, speak.
5: 
6: First Citizen:
7: You are all resolved rather to die than to famish?
8: 
9: All:
10: Resolved. resolved.
11: 
12: First Citizen:
13: First, you know Caius Marcius is chief enemy to the people.
14: 

--- Filtered Corpus (first 10 sentences) ---
0: <start> before we proceed any further , hear me speak . <end>
1: <start> speak , speak . <end>
2: <start> you are all resolved rather to die than to famish ? <end>
3: <start> resolved . resolved . <end>
4: <start> first , you know caius marcius is chief enemy to the people . <end>
5: <start> we know t , we know t . <end>
6: <start> let us kill him , and we ll have corn at our own price . <end>
7: <start> is t a verdict ? <end>
8: <start> no more talking on t let it be done away , away ! <end>
9: <start> one word , good citizens . <end>


## Summary:

### Data Analysis Key Findings

*   The `raw_corpus` was successfully loaded from `shakespeare.txt`, containing raw text including empty lines and lines ending with colons as expected.
*   The `corpus` was accurately generated by filtering out empty lines and sentences ending with a colon from the `raw_corpus`.
*   Beyond the initial filtering, the `corpus` also underwent additional preprocessing steps, including lowercasing all text and appending `<start>` and `<end>` tokens to each sentence, indicating readiness for model training.

### Insights or Next Steps

*   The successful initial data loading and preprocessing have prepared a clean and structured dataset, which is crucial for the subsequent development and training of the Shakespeare-style AI writing model.
*   The immediate next step would involve further preparing this `corpus` for the AI model, likely through tokenization and creation of input-output sequence pairs, before proceeding to model architecture definition and training.


In [26]:
import tensorflow as tf
import os
import re
import glob

print("텐서플로우 버전:", tf.__version__) # 도구 버전 확인!


텐서플로우 버전: 2.19.0


In [27]:
# 파일 경로 설정 (여러분의 경로에 맞게 수정이 필요할 수 있어요!)
# 보통 실습 환경에서는 아래 경로를 많이 써요.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'

# 파일이 없다면 예시 데이터를 다운로드 받는 코드 (혹시 파일이 없을까봐 준비했어요)
if not os.path.exists(file_path):
    path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
    file_path = path_to_file

# 데이터 읽어서 리스트에 담기
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

print("데이터 크기:", len(raw_corpus))
print("앞부분 5줄만 읽어볼까요?\n", raw_corpus[:5])

# 빈 줄이나 너무 짧은 줄은 버리고 'corpus'라는 깨끗한 바구니에 담아요
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    corpus.append(sentence)


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
데이터 크기: 40000
앞부분 5줄만 읽어볼까요?
 ['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.']


In [28]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1. 소문자로 바꾸고 양쪽 공백 삭제
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2. 특수문자 양쪽에 공백 넣기
    sentence = re.sub(r'[" "]+', " ", sentence) # 3. 여러 공백은 하나로
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4. 영어랑 특수문자만 남기기
    sentence = sentence.strip() # 5. 다시 양쪽 공백 삭제
    sentence = '<start> ' + sentence + ' <end>' # 6. 시작과 끝 표시 붙이기
    return sentence

# 청소 도구로 문장들을 싹싹 닦아서 정제된 문장들만 모아요
processed_corpus = []
for sentence in corpus:
    processed_corpus.append(preprocess_sentence(sentence))

print("청소 전:", corpus[0])
print("청소 후:", processed_corpus[0])


청소 전: Before we proceed any further, hear me speak.
청소 후: <start> before we proceed any further , hear me speak . <end>


In [29]:
def tokenize(corpus):
    # 7000단어만 기억하는 토크나이저 만들기
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus) # 단어장 만들기
    tensor = tokenizer.texts_to_sequences(corpus) # 글자를 숫자로 변환

    # 문장 길이를 20으로 맞추기 (짧으면 뒤를 0으로 채움)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=20)

    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(processed_corpus)

print("변환된 데이터 모양:", tensor.shape)
print("단어장 2번은?:", tokenizer.index_word[2])


[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras.src.legacy.preprocessing.text.Tokenizer object at 0x7df746dbc740>
변환된 데이터 모양: (24015, 20)
단어장 2번은?: <start>


In [30]:
# 문제(src): 끝 부분(<end>) 제외
src_input = tensor[:, :-1]
# 정답(tgt): 시작 부분(<start>) 제외
tgt_input = tensor[:, 1:]

print("문제 예시:", src_input[0])
print("정답 예시:", tgt_input[0])

# 텐서플로우 데이터셋으로 포장하기 (학습하기 좋게 묶음)
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


문제 예시: [  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0]
정답 예시: [143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0]


In [31]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # 1. 단어를 벡터로 만드는 층
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        # 2. 문장을 기억하고 이해하는 LSTM 층 (두 개나 써요!)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        # 3. 다음 단어를 예측하는 층
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        return out

# 모델 생성!
embedding_size = 256
hidden_size = 1024
vocab_size = 7000 + 1 # 단어장 크기 + 1 (패딩용 0번 포함)

model = TextGenerator(vocab_size, embedding_size, hidden_size)

# 모델에게 한 번 시범 보여줘서 구조 확정하기
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

model.summary() # 모델 구조 확인


Model: "text_generator_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (256, 19, 256)         │     1,792,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (256, 19, 1024)        │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (256, 19, 1024)        │     8,392,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (256, 19, 7001)        │     7,176,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,607,961 (86.24 MB)

 Trainable params: 22,607,961 (86.24 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

print("학습 시작! 조금 오래 걸릴 수 있어요 ☕")
model.fit(dataset, epochs=30)


학습 시작! 조금 오래 걸릴 수 있어요 ☕
Epoch 1/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 36s 373ms/step - loss: 4.3884
Epoch 2/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 33s 354ms/step - loss: 2.8950
Epoch 3/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 33s 360ms/step - loss: 2.7407
Epoch 4/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 360ms/step - loss: 2.6474
Epoch 5/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 33s 358ms/step - loss: 2.5779
Epoch 6/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 361ms/step - loss: 2.5266
Epoch 7/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 361ms/step - loss: 2.4746
Epoch 8/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 33s 358ms/step - loss: 2.4242
Epoch 9/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 362ms/step - loss: 2.3721
Epoch 10/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 362ms/step - loss: 2.3280
Epoch 11/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 33s 359ms/step - loss: 2.2612
Epoch 12/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 33s 359ms/step - loss: 2.2155
Epoch 13/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 34s 363ms/step - loss: 2.1680
Epoch 14/30
93/93 ━━━━━━━━━━━━━━━━━━━━ 33s 359ms/step - loss: 2.1069
Epoch 15/30
93/93 ━

In [33]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트용 입력 문장 전처리
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        # 1. 다음 단어 예측
        predict = model(test_tensor)
        # 2. 가장 확률 높은 단어 선택
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]

        # 3. 예측한 단어를 문장에 붙이기
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 4. <end>가 나오거나 문장이 길어지면 끝
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 숫자를 다시 글자로 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

# 테스트 해보기!
print("생성된 문장:", generate_text(model, tokenizer, init_sentence="<start> he"))
print("생성된 문장:", generate_text(model, tokenizer, init_sentence="<start> i love"))


생성된 문장: <start> he is , and feasts the nobles of the state <end> 
생성된 문장: <start> i love you ll not , i am sure of breath <end> 


In [34]:
# 테스트하고 싶은 문장들을 리스트에 담았어요
test_sentences = [
    "<start> i love",          # 사랑 고백
    "<start> the king",        # 왕에 대한 이야기
    "<start> why do you",      # 의문문
    "<start> shall i compare", # 셰익스피어의 유명한 소네트 말투
    "<start> to be or not",    # 햄릿의 명대사 흉내내기
    "<start> my heart",        # 감성적인 표현
]

print("-" * 50)
print("📜 AI 셰익스피어의 작문 실력 테스트 📜")
print("-" * 50)

for sentence in test_sentences:
    # 우리가 만든 generate_text 함수로 문장 생성하기
    generated_line = generate_text(model, tokenizer, init_sentence=sentence, max_len=20)

    # 보기 좋게 출력하기
    print(f"질문: {sentence.replace('<start>', '').strip()}...") # <start>는 빼고 보여주기
    print(f"작문: {generated_line}")
    print("-" * 50)


--------------------------------------------------
📜 AI 셰익스피어의 작문 실력 테스트 📜
--------------------------------------------------
질문: i love...
작문: <start> i love you ll not , i am sure of breath <end> 
--------------------------------------------------
질문: the king...
작문: <start> the king s <unk> on the earth of york <end> 
--------------------------------------------------
질문: why do you...
작문: <start> why do you think thwack it ? <end> 
--------------------------------------------------
질문: shall i compare...
작문: <start> shall i compare your grace ? <end> 
--------------------------------------------------
질문: to be or not...
작문: <start> to be or not a wife , and that s past , <end> 
--------------------------------------------------
질문: my heart...
작문: <start> my heart is not confederate with my hand , <end> 
--------------------------------------------------
